In [3]:
import pandas as pd

def cargar_archivos(datos_csv, olympics_csv):
    # Cargar el archivo CSV "country-codes.csv", el cual servirá para la homologación de países en ambos archivos. En el caso de los datos olímpicos, los códigos de los países están en formato IOC
    df_country_codes = pd.read_csv(datos_csv)

    # Cargar el archivo CSV "olympics_games.csv", el cual contiene datos históricos de los competidores de los juegos olímpicos desde 1896 a 2016
    df_olympics = pd.read_csv(olympics_csv)
    
    return df_country_codes, df_olympics

def filtrar_olympics(df_olympics, df_country_codes):
    # Realizar un merge entre los dos dataframes utilizando la columna NOC y IOC del archivo de códigos
    df_merged = pd.merge(df_olympics, df_country_codes, left_on='NOC', right_on='IOC')

    # Convertir la columna 'Year' a tipo fecha
    df_merged['Year'] = pd.to_datetime(df_merged['Year'], format='%Y')

    # Filtrar los datos a partir del año 1960, ya que los datos del Banco Central abarcan desde 1960 a 2016 
    df_merged = df_merged[df_merged['Year'] >= '1960']

    return df_merged

def guardar_tabla_olympics(df_filtered):
    # Filtrar las columnas necesarias
    df_filtered = df_filtered[['Country Name', 'Year', 'Season', 'City']]

    # Agrupar los datos por país, año, temporada y ciudad, y contar los participantes. Esto también se puede hacer directamente en Tableau, pero de esta manera se mantiene un mejor control de los datos
    df_grouped = df_filtered.groupby(['Country Name', 'Year', 'Season', 'City']).size().reset_index(name='Cantidad de Participantes')

    # Guardar el DataFrame resultante en un archivo CSV que servirá para crear el gráfico de competidores por país para cada año.
    df_grouped.to_csv('tabla_olympics.csv', index=False)

def filtrar_medallas(df_merged):
    # Filtrar las columnas necesarias
    df_filtrado2 = df_merged[['Country Name', 'Year', 'Sport', 'Medal']]

     # Agrupar los datos por país, año, deporte y tipo de medalla, y contar las medallas
    df_agrupado2 = df_filtrado2.groupby(['Country Name', 'Year', 'Sport', 'Medal']).size().reset_index(name='Total Medals')

    # Crear un nuevo archivo csv con los datos del dataframe que servirá para crear los gráficos de medallas obtenidas por cada país
    df_agrupado2.to_csv('medallas_año.csv', index=False)

def main(datos_csv, olympics_csv):
    df_country_codes, df_olympics = cargar_archivos(datos_csv, olympics_csv)
    df_merged = filtrar_olympics(df_olympics, df_country_codes)
    guardar_tabla_olympics(df_merged)
    filtrar_medallas(df_merged)

if __name__ == "__main__":
    main('datos.csv', 'olympics_games.csv')

